In [1]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path
from pydantic import BaseModel, validator, Field

datafile = Path("merry20211207103305.output.json")

# Read JSON file
with open(datafile, "r") as f:
    data = json.load(f)


class Effect(BaseModel):
    starttime: str = Field(alias="start_time")
    endtime: str = Field(alias="end_time")
    label: str = Field(alias="content")

    @validator("starttime", "endtime", pre=True)
    def check_time(cls, v):
        return str(int(float(v) * 1000))


class Effects(BaseModel):
    effects: list[Effect]


In [2]:
words = []
for i in data["results"]["items"]:
    if i["type"] != "pronunciation":
        continue

    if len(words) == 0:
            this_word = Effect(
                content="",
                start_time=0,
                end_time=i["start_time"],
            )
            words.append(this_word)
    elif words[-1].endtime != i["start_time"]:
            this_word = Effect(
                content="",
                start_time=int(words[-1].endtime) / 1000,
                end_time=i["start_time"],
            )
            words.append(this_word)


    this_word = Effect(
        content=i["alternatives"][0]["content"],
        start_time=i["start_time"],
        end_time=i["end_time"],
    )


    # print(this_word)
    # effect = Effect(**this_word)
    words.append(this_word)

In [3]:
len(words)

262

In [4]:
words[0]

Effect(starttime='0', endtime='1340', label='')

In [5]:
words[-1]

Effect(starttime='39740', endtime='39940', label='Yeah')

In [6]:
effects = Effects(effects=words)

<?xml version="1.0" encoding="UTF-8"?>
<timings>
    <timing name="New Timing" SourceVersion="2018.4">
        <EffectLayer>
            <Effect label="It\'s" startTime="240" endTime="12270" />

In [7]:
root = ET.Element("timings")
timing = ET.SubElement(root, "timing", attrib={"name": "New Timing", "SourceVersion": "2018.4"})
effectlayer = ET.SubElement(timing, "EffectLayer")
for word in words:
    ET.SubElement(effectlayer, "Effect", attrib=word.dict())

In [9]:
tree = ET.ElementTree(root)
tree.write("filename.xml")

In [8]:
ET.dump(root)

<timings><timing name="New Timing" SourceVersion="2018.4"><EffectLayer><Effect starttime="0" endtime="1340" label="" /><Effect starttime="1340" endtime="1860" label="merry" /><Effect starttime="1860" endtime="1860" label="" /><Effect starttime="1860" endtime="2480" label="christmas" /><Effect starttime="2480" endtime="2480" label="" /><Effect starttime="2480" endtime="2590" label="and" /><Effect starttime="2590" endtime="2590" label="" /><Effect starttime="2590" endtime="2700" label="we" /><Effect starttime="2700" endtime="2700" label="" /><Effect starttime="2700" endtime="2920" label="hope" /><Effect starttime="2920" endtime="2920" label="" /><Effect starttime="2920" endtime="3050" label="you're" /><Effect starttime="3050" endtime="3050" label="" /><Effect starttime="3050" endtime="3440" label="enjoying" /><Effect starttime="3440" endtime="3440" label="" /><Effect starttime="3440" endtime="3530" label="the" /><Effect starttime="3530" endtime="3530" label="" /><Effect starttime="3530" 